In [11]:
import re
import numpy as np
import math
import torch
from random import *
from torch import nn
from torch import optim
from torch.utils import data

In [ ]:
text = (
    'Hello, how are you? I am Romeo.\n' # R
    'Hello, Romeo My name is Juliet. Nice to meet you.\n' # J
    'Nice meet you too. How are you today?\n' # R
    'Great. My baseball team won the competition.\n' # J
    'Oh Congratulations, Juliet\n' # R
    'Thank you Romeo\n' # J
    'Where are you going today?\n' # R
    'I am going shopping. What about you?\n' # J
    'I am going to visit my grandmother. she is not very well' # R
)
sentences = re.sub("[.,!?\\-]",' ',text.lower()).split('\n')
print(np.array(sentences).shape)

In [ ]:
word_list = list(set(" ".join(sentences).split()))
word_2_index = {'[PAD]':0,'[CLS]':1,'[SEP]':2,'[MASK]':3}
#构建语料库
for i,w in enumerate(word_list):
    word_2_index[w] = i + 4
index_2_word = {i: w for i,w in enumerate(word_2_index)}
vocab_size = len(word_2_index)
print(index_2_word)

In [ ]:
token_list = list()
for sentence in sentences:
    arr = [word_2_index[s] for s in sentence.split()]
    token_list.append(arr)

In [ ]:
maxlen = 30
batch_size = 6
max_pred = 5 # max tokens of prediction
n_layers = 6
n_heads = 12
d_model = 768
d_ff = 768*4 # 4*d_model, FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_segments = 2

In [31]:
def make_data():
    batch = []
    positive = negative = 0
    while positive!=batch_size/2 or negative!=batch_size/2:
        tokens_a_index, tokens_b_index = randrange(len(sentences)),randrange(len(sentences))
        tokens_a,tokens_b = token_list[tokens_a_index],token_list[tokens_b_index]

        # cls的index + token_A句子 + sep的index + token_B 句子 + sep的index
        input_ids = [word_2_index['[CLS]']]+tokens_a+[word_2_index['[SEP]']]+tokens_b+[word_2_index['[SEP]']]


        # CLS(0)+第一句[0]+SEP(0)+第二句[1]+SEP(1)
        segment_ids = [0] * (1 + len(tokens_a)+ 1) + [1] * (len(tokens_b) + 1)

        # MASK LM
        n_pred = min(max_pred,max(1,int(len(input_ids)*0.15)))

        # 候选mask的位置
        cand_maked_pos = [i for i,token in enumerate(input_ids) if token!=word_2_index['[SEP]'] and token!=word_2_index['[CLS]'] ]

        shuffle(cand_maked_pos)

        masked_tokens, masked_pos = [], []

        for pos in cand_maked_pos[:n_pred]:
            masked_pos.append(pos)
            #input_id 对应位置的值就是token
            masked_tokens.append(input_ids[pos])
            if random()<0.8:
                input_ids[pos] = word_2_index['[MASK]']
            elif random()>0.9:
                index = randint(0, vocab_size-1)

                while index<4:
                    index = randint(0, vocab_size - 1)
                input_ids[pos] = index

        #零填充
        #未达到最大长度的进行填充
        n_pad = maxlen - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)

        # Zero Padding (100% - 15%) tokens
        if max_pred > n_pred:
            n_pad = max_pred - n_pred
            masked_tokens.extend([0] * int(n_pad))
            masked_pos.extend([0] * int(n_pad))

        if tokens_a_index+1 == tokens_b_index and positive<batch_size/2:
            batch.append([input_ids,segment_ids,masked_tokens,masked_pos,True])
            positive +=1
        elif tokens_a_index+1 !=tokens_b_index and negative<batch_size/2:
            batch.append([input_ids,segment_ids,masked_tokens,masked_pos,False])
            negative +=1

    return batch

In [32]:
batch = make_data()
input_ids, segment_ids, masked_tokens, masked_pos, isNext = zip(*batch)
input_ids, segment_ids, masked_tokens, masked_pos, isNext = \
    torch.LongTensor(input_ids),  torch.LongTensor(segment_ids), torch.LongTensor(masked_tokens),\
    torch.LongTensor(masked_pos), torch.LongTensor(isNext)


In [34]:
class myDataset(data.Dataset):
    def __init__(self,input_ids, segment_ids, masked_tokens, masked_pos, isNext):
        self.input_ids = input_ids
        self.segment_ids = segment_ids
        self.masked_tokens  = masked_tokens
        self.masked_pos = masked_pos
        self.isNext= isNext
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, index):
        return self.input_ids[index], self.segment_ids[index], self.masked_tokens[index], self.masked_pos[index], self.isNext[index]

loader = data.DataLoader(myDataset(input_ids,segment_ids,masked_tokens,masked_pos,isNext),batch_size,True)
loader